In [58]:
from bs4 import BeautifulSoup
import re


class HTMLCorrector:
    def __init__(self, path):
        """ This class represents the HTML code corrector . The purpose of this class is format the .htm file 
         according to the reference pdf generated (by Adobe Framemaker). 
        
        Args:
        - path: Path of the generated .htm file by Framemaker
        - header: A string that would store the header for correct HTML file
        
        """
        self.path = path
        self.header="" ## This will be updated_elementsted, once we will read the .HTML file generated by Framemaker
    
    def __read_file(self):
        """ Function to head the .HTML file and convert it into a string 
        
        Args:
        - path: Path of the generated .htm file by Framemaker
        
        Output:
        - content: A string that has the unformatted HTML code.
        """
        code=""
        with open(self.path, "r") as f:
            for line in f.readlines():            
                code =code+line
        return code
    
    
    def __add_pagenumbers(self,header):
          """ Function to add pagenumbers to the HTML file 
        
        Args:
        - header: Header portion of the HTML file
        
        Output:
        - updated_header: Header with embedded javascript code for page numbers
        """
        with open ("PNumberScript.js", "r") as myfile:
            JSCODE=myfile.readlines()
        start= """<script type="text/javascript">window.onload = addPageNumbers;"""
        end = "</script>"
        
        htmlcode =  start + '\n'.join(JSCODE) +end
        updated_header = header.replace("<head>","<head>"+htmlcode)
        return updated_header
    
    
    
    def __add_padding(self,x):
         """ Function to add paddings to the HTML file
        
        Args:
        - x: content of the HTML file
        
        Output:
        - x_updated: content with padding of 5%
        """
        
        x = x.replace("<body>","<body>"+"""<div style="padding:5%;">""")
        x_updated = x.replace("</body>","</div>"+"</body>")
        return x_updated
        
        
        
    def __add_header(self,x):
        """ Function to add header in our corrected HTML file 
        
        Args:
        - x: code without header 
        
        Output:
        - code_with_header: A string that has the HTML code along with the header.
        """
        si = x.find('<head>')
        ei = x.find('</head>')
        self.header = self.__add_pagenumbers(self.header)
        code_with_header = x[0:si-1]  + self.header + x[ei+7:]
        return code_with_header
    
        
   
    def __format_caption(self,temp):        
        """ This function will change the formatting of caption. It performs the following operations:
            1) Remove the </br> tag in between caption and text
            2) Remove </p> tag to avoid the next line
            3) Bolding the caption
        
        Args:
        - temp: HTML code string for caption tag. 
        
        Output:
        - f_string: A string that has the formatted HTML code for caption.
        """
        temp = (temp.replace('<br/>',''))
        temp = (temp.replace('</p>',''))
        f_string = temp.split('\n')
        f_string.insert(1, '<Strong>') 
        f_string.append('</Strong>')
        return ((''.join(f_string))) 
    
    
    def __format_text(self,temp):        
        """ This function will change the formatting of caption. It performs the following operations:
            1) Remove the <p> tag in between caption and text
            2) Split the string to get the required part of it.
        
        Args:
        - temp: HTML code string for text tag. 
        
        Output:
        - f_string: A string that has the formatted HTML code for text.
        """
        f_string = temp.split('\n')
        return ((''.join(f_string[1:])))
    
    def __change_finger_image(self,tag):
        """
        The function will change the image of the bullet points.

        Args:
        - tag: string of <p> tag element
        
        Output:
        - tag: Updated string that has the correct code for <p> tag element .
        """
        
        if bool(re.search("FingerParInd", tag)):
            tag = " ".join(re.split("\s+", tag, flags=re.UNICODE))
            tag =tag.replace("""<span class="FingerSolid"> * </span>""", """<span class="FingerSolid"> <img src="assets/FingerParInd.png" style="width:1.1rem;height:0.8rem;" > </span>""")          
            return tag
        elif bool(re.search("Finger2dParInd", tag)) or bool(re.search("Finger3dParInd", tag))  :
            tag = " ".join(re.split("\s+", tag, flags=re.UNICODE))
            tag =tag.replace("""<span class="FingerSolid"> + </span>""", """<span class="FingerSolid"> <img src="assets/Finger2dParInd.png" style="width:1.1rem;height:0.8rem;" > </span>""")          
            return tag
        else:
            return tag
        
    
    def __change_bullet_image(self,tag):
        """
        The function will change the image of the bullet points.

        Args:
        - tag: string of <p> tag element
        
        Output:
        - tag: Updated string that has the correct code for <p> tag element .
        """
        
        if bool(re.search("SquareHollowBullets", tag)) or bool(re.search("SquareBullets", tag)):
            tag = " ".join(re.split("\s+", tag, flags=re.UNICODE))
            tag =tag.replace("""<span class="SquareHollowBullets"> o </span>""", """<span class="SquareHollowBullets"> <img src="assets/SquareHollowBullets.png" style="width:0.8rem;height:0.8rem;" > </span>""")
            tag =tag.replace("""<span class="SquareBullets"> n </span>""", """<span class="SquareBullets"> <img src="assets/SquareBullets.png" style="width:0.8rem;height:0.8rem;" > </span>""")
            
            return tag
        else:
            return tag
        
    def __indent_bullets(self,tag):
        """
        The function will correct the class of the bullet points with wrong indentation level

        Args:
        - tag: string of <p> tag element
        
        Output:
        - tag: Updated string that has the correct code for <p> tag element .
        """
        if bool(re.search("SquareHollowBullets", tag)) and bool(re.search("BulletPar2dInd", tag)):
            tag = tag.replace("BulletPar2dInd","Bullet2dPar")
            return tag
        else:
            return tag
        
    def __bold_example_caption(self,tag):
        """
        The function will correct the font-style of Example captions

        Args:
        - tag: string of <p> tag element
        
        Output:
        - tag: Updated string that has the correct code for <p> tag element .
        """
        r ="Example.*:"
        repl = "<strong>" + re.findall(r, tag)[0] +"</strong>"
        tag = re.sub(r, repl, tag)
        return tag
    
    def beautify(self,content,indent_level=4): 

        """ The .HTML file generated by Framemaker has un formatted code. This function uses HTMLBeautifier to
        beautify the code.

        Args:
        - content: unformatted HTML code string. 
        - indent_level: Indentation level of the beatufied code. 

        Output:
        - beautified_code: A string that has the beatufied HTML code.
        """
        assert indent_level > 0, f"Indent level must be greater than zero. Got: {indent_level}"

        from html5print import HTMLBeautifier
        beautified_code= (HTMLBeautifier.beautify(content, indent_level))
        return beautified_code

    
    
    def correctify_code(self):
        """ The function will perform all the operations needed to correct the format of the
        HTML code.
        
        Output:
        - corrected_code: A string that has the code after applying the 
                          pre-processing operations.
        """
        file = self.__read_file() ## Reading the file
        b_code = self.beautify(file) ## Beautifying it
        pair_check=False 
        soup = BeautifulSoup(b_code)
        all_elements = soup.findAll('p')
        self.header = str(soup.findAll('head')[0])
        
        
        updated_elements=[]
        for i in range(len(all_elements)):

            element = str(all_elements[i]) ## Converting div type to string
            temp = "".join([s for s in element.strip().splitlines(True) if s.strip()]) ## Remving lines with no text
            temp = self.__indent_bullets(temp) ## Correcting indentation of bullets
            temp = self.__change_bullet_image(temp) ## Correcting images of bullets
            temp = self.__change_finger_image(temp) ## Correcting images of fingers
            ## Patterns for detection of caption and Text
            regex_caption = r'_[A-Za-z0-9]+Capt'
            regex_textI = r'_[A-Za-z0-9]+Text'
            regex_textII = r'_[A-Za-z0-9]+SecondPar'
            regex_example ="Example.*:"
            
            
            btemp =self.beautify(temp)     
            if bool(re.search(regex_caption, temp)) or bool(re.search("_Q1", temp)) or bool(re.search("_R1", temp)):
                updated_elements.append(self.__format_caption(temp))
                pair_check=True

            elif ( bool(re.search(regex_textI, temp)) or bool(re.search(regex_textII, temp)) or bool(re.search("_R2", temp)) or bool(re.search("_Q2", temp)) ) and pair_check:
                updated_elements.append(self.__format_text(temp))
                pair_check=False
            
            elif bool(re.search(regex_example, btemp)):
                updated_elements.append(self.__bold_example_caption(btemp))
            else:
                updated_elements.append(temp)
        corrected_code = self.beautify(''.join(updated_elements), 4)
        ## Removing gap between caption and text
        corrected_code = corrected_code.replace('&nbsp;','')
        return corrected_code
    
    
    
    def generate_correct_file(self,filename):
        """
        The function will correct the class of the bullet points with wrong indentation level

        Args:
        - filename: Name of the updated file 
        
        Output:
        - An output file with name as "filename" (string) in the present working directory
        """
        result = self.correctify_code()
        f_result = self.__add_header(result)
        f_result = self.__add_padding(f_result)
        
        
        with open(filename, "w") as text_file:
            text_file.write(f_result)
        
        

# main func
if __name__ == '__main__':
    Filename = './Ch14.htm' ## Path of the file generated by Frame maker
    Updated_File = "Formatted_Ch14.html" ## Name of the output file 
    HTMLCorrector(Filename).generate_correct_file(Updated_File)         
    
        
